<a href="https://colab.research.google.com/github/team0243/Project_ML/blob/main/Image_classification_RCC_Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import PIL

# Preparing The Data


In [ ]:
# Define data transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the data directory
data_dir = Path("/content/drive/MyDrive/โครงการFF/Resnet-18")

# Create data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#image_datasets

In [ ]:
print(data_dir)

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
class_names

In [ ]:
# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Use all parameters


# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")

In [ ]:
# Save the model
torch.save(model.state_dict(), 'RCC_classification_model.pth')

## Classification on Unseen Image
To use the saved model to classify unseen images, you need to load the model and then apply it to the new images for inference.

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

# Load the saved model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1000)  # Adjust to match the original model's output units
model.load_state_dict(torch.load('/content/RCC_classification_model.pth'))
model.eval()

# Create a new model with the correct final layer
new_model = models.resnet18(pretrained=True)
new_model.fc = nn.Linear(new_model.fc.in_features, 2)  # Adjust to match the desired output units

# Copy the weights and biases from the loaded model to the new model
new_model.fc.weight.data = model.fc.weight.data[0:2]  # Copy only the first 2 output units
new_model.fc.bias.data = model.fc.bias.data[0:2]

Prepare your new image for classification. You should use the same data transformations you used during training. Here's an example of how to prepare an image for inference:

test ccRCC images

In [ ]:
# Load and preprocess the unseen image
image_path = '/content/A064.jpg'  # Replace with the path to your image
image = Image.open(image_path)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

Perform inference using the model:

In [ ]:
import os

# Directory containing the images you want to classify
image_dir = '/content/drive/MyDrive/โครงการFF/Images_Classification/Test'  # Replace with the actual path

# List all image files in the directory
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Loop through the images and classify each one
for image_path in image_files:
    # Load and preprocess the image
    image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        output = model(input_batch)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    predicted_class_index = torch.argmax(probabilities).item()
    predicted_probability = probabilities[predicted_class_index].item()
    predicted_class_name = class_names[predicted_class_index]

    print(f'Image: {image_path}')
    print(f'The predicted class is: {predicted_class_name}')
    print(f'The predicted probability is: {predicted_probability:.4f}')

    # Display the image (optional)
    image = np.array(image)
    plt.imshow(image)
    plt.axis('off')
    plt.text(10, 10, f'Predicted: {predicted_class_name} ({predicted_probability:.2%})', fontsize=12, color='white', backgroundcolor='red')
    plt.show()


In [ ]:
# prompt: Create Evaluating The Model

# Evaluating the Model
def evaluate_model(model, dataloader, criterion, device):
  """Evaluates the model on a given dataloader.

  Args:
    model: The PyTorch model to evaluate.
    dataloader: The PyTorch DataLoader for the evaluation dataset.
    criterion: The loss function.
    device: The device to run the evaluation on (e.g., 'cuda' or 'cpu').

  Returns:
    A tuple containing the average loss and accuracy.
  """
  model.eval()
  running_loss = 0.0
  running_corrects = 0
  total_samples = 0

  with torch.no_grad():
    for inputs, labels in dataloader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
      total_samples += inputs.size(0)

  epoch_loss = running_loss / total_samples
  epoch_acc = running_corrects.double() / total_samples

  return epoch_loss, epoch_acc


# Evaluate the model on the validation set
val_loss, val_acc = evaluate_model(model, dataloaders['val'], criterion, device)
print(f'Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}')


In [ ]:
# prompt: create Evaluating The Model  Accuracy

def evaluate_model(model, dataloader, criterion, device):
  """Evaluates the model on a given dataloader.

  Args:
    model: The PyTorch model to evaluate.
    dataloader: The PyTorch DataLoader for the evaluation dataset.
    criterion: The loss function.
    device: The device to run the evaluation on (e.g., 'cuda' or 'cpu').

  Returns:
    A tuple containing the average loss and accuracy.
  """
  model.eval()
  running_loss = 0.0
  running_corrects = 0
  total_samples = 0

  with torch.no_grad():
    for inputs, labels in dataloader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
      total_samples += inputs.size(0)

  epoch_loss = running_loss / total_samples
  epoch_acc = running_corrects.double() / total_samples

  return epoch_loss, epoch_acc

# Evaluate the model on the validation set
val_loss, val_acc = evaluate_model(model, dataloaders['val'], criterion, device)
print(f'Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}')


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Initialize lists to store true labels and predicted labels
y_true = []
y_pred = []

# Iterate through the validation set
for inputs, labels in dataloaders['val']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

    y_true.extend(labels.cpu().numpy().tolist())
    y_pred.extend(preds.cpu().numpy().tolist())

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

# Calculate and print the classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:")
print(report)
